In [ ]:
import numpy as np
import glob
from PIL import Image
import os

In [ ]:
ls


In [ ]:
ls training_set/

In [ ]:
C = glob.glob('train/cats/*')
D = glob.glob('train/dogs/*')

In [ ]:
#read cats images
CX = []
CY = [0]*len(C)
for i in C:
    CX.append(np.array(Image.open(i).convert('L')))

CX = np.array(CX).reshape(np.array(CX).shape[0],-1)
CY = np.array(CY).reshape(-1,1)
print(CX.shape)
print(CY.shape)

In [43]:
#read dogs image
DX = []
DY = [0]*len(D)
for i in D:
    DX.append(np.array(Image.open(i).convert('L')))

DX = np.array(DX).reshape(np.array(DX).shape[0],-1)
DY = np.array(DY).reshape(-1,1)
DY.shape

(80, 1)

In [44]:
#join dogs and cats images
X = np.concatenate((CX,DX))
Y = np.concatenate((CY,DY))
print(X.shape, Y.shape)

(160, 1) (160, 1)


In [56]:
X

In [45]:
import numpy as np
import matplotlib.pyplot as plt
import math, time

In [46]:
class TwoLayerNet():
    def __init__(self, input_size, hidden_size, output_size, std=1e-4):
        self.params = {}
        self.params['W1'] = std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    


In [47]:
    def loss(self, X, y=None, reg=0.0):
        W1, b1 = self.params['W1'], self.params['b1']
        W2, b2 = self.params['W2'], self.params['b2']
        N, D = X.shape
        
        # Compute the forward pass
        scores = None
    
        # Values of the hidden layer
    
        X1 = np.maximum( X.dot(W1) + b1, 0 )
        X2 = X1.dot(W2) + b2
        
        scores = X2
        
        if y is None:
          return scores
   
        # Creating a output matrix from output vector
        # X2 is the output
        Y = np.zeros(X2.shape)
        for i,row in enumerate(Y):
            row[y[i]] = 1
        
        loss = 0.0
        
        #Exponentiating the X2 , then normalizing the exponentiated value
        exp_X2 = pow(math.e, X2)
        scores   = (exp_X2.T/np.sum(exp_X2, axis = 1)).T
    
        for i in range(N):
            loss -= np.log( scores[i][y[i]] )
        loss /= N
        loss += 0.5*reg*np.sum(W1 * W1)  
        loss += 0.5*reg*np.sum(W2 * W2) 
        
        grads = {}
        
        
        #Back propagation
        dX2 = scores - Y
        dW2 = ((dX2.T).dot(X1)).T/N + reg*W2
        db2 = (dX2.T).dot(np.ones(X1.shape[0]))/N
        dX1 = dX2.dot(W2.T)
        dW1 = (((dX1*(X1>0)).T).dot(X)).T/N + reg*W1
        db1 = (((dX1*(X1>0)).T).dot(np.ones(X.shape[0]))).T/N
      
        grads['W2'] = dW2
        grads['b2'] = db2
        grads['W1'] = dW1
        grads['b1'] = db1
        
        return loss, grads

In [48]:
    def train(self, X, y, X_val, y_val,
              learning_rate=1e-3, learning_rate_decay=0.95,
              reg=1e-5, num_iters=100,batch_size=200, 
              verbose=False ,dropout_fraction = 0):
        
        Binom_variables = np.random.choice([0, 1], size=X.shape, p=[dropout_fraction,1-dropout_fraction])
        X = (X*Binom_variables)/(1-dropout_fraction)
        
        
        num_train = X.shape[0]
        iterations_per_epoch = max(num_train / batch_size, 1)
        val_acc = 0

        # Use SGD to optimize the parameters in self.model
        loss_history = []
        train_acc_history = []
        val_acc_history = []
        
        for it in xrange(num_iters):
            batch_indicies = np.random.choice(num_train, batch_size, replace = False)
            X_batch = X[batch_indicies]
            y_batch = y[batch_indicies]
            
            # Compute loss and gradients using the current minibatch
            loss, grads = self.loss(X_batch, y=y_batch, reg=reg)
            loss_history.append(loss)
        
            for variable in self.params:
                self.params[variable] -= learning_rate*grads[variable]
            
            if verbose and it % 100 == 0:
                val_acc = (self.predict(X_val) == y_val).mean()
                print ('iteration %d / %d: loss %f, val_acc %f' % (it, num_iters, loss, val_acc))
            
            # Every epoch, check train and val accuracy and decay learning rate.
            if it % iterations_per_epoch == 0:
                # Check accuracy
                train_acc = (self.predict(X_batch) == y_batch).mean()
                val_acc = (self.predict(X_val) == y_val).mean()
                train_acc_history.append(train_acc)
                val_acc_history.append(val_acc)
                # Decay learning rate
                learning_rate *= learning_rate_decay
        
        return {
            'loss_history': loss_history,
            'train_acc_history': train_acc_history,
            'val_acc_history': val_acc_history,
        }

In [49]:
    def predict(self, X):
        y_pred = None
        X1 = np.maximum( X.dot(self.params['W1']) + self.params['b1'], 0 )
        X2 = X1.dot(self.params['W2']) + self.params['b2']
        exp_X2 = pow(math.e, X2)
        scores   = (exp_X2.T/np.sum(exp_X2, axis = 1)).T
    
        y_pred = np.argmax(scores, axis = 1)
        
        return y_pred

In [51]:
CandD = TwoLayerNet(100,2,0)
CandD.train(X, Y, X, Y)

AttributeError: 'TwoLayerNet' object has no attribute 'train'